<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_NRPy`: Solving the Induction Equation

## Author: Patrick Nelson

**Notebook Status:** <font color='green'><b>Validated</b></font>

**Validation Notes:** This code has been validated by showing that it converges to the exact answer at the expected order.

### NRPy+ Source Code for this module:
* [GiRaFFE_NRPy/GiRaFFE_NRPy_Afield_flux_handwritten.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Afield_flux_handwritten.py)

## Introduction

This notebook documents the function from the original `GiRaFFE` that calculates the flux for $A_i$ according to the method of Harten, Lax, von Leer, and Einfeldt (HLLE), assuming that we have calculated the values of the velocity and magnetic field on the cell faces according to the piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf), modified for the case of GRFFE. 

Our goal in this module is to write the code necessary to solve the induction equation 
$$
\partial_t A_i = \underbrace{\epsilon_{ijk} v^j B^k}_{\rm Flux\ terms} - \underbrace{\partial_i \left(\alpha \Phi - \beta^j A_j \right)}_{\rm Gauge\ terms}.
$$
To properly handle the flux terms and avoid problems with shocks, we cannot simply take a cross product of the velocity and magnetic field at the cell centers. Instead, we must solve the Riemann problem at the cell faces using the reconstructed values of the velocity and magnetic field on either side of the cell faces. The reconstruction is done using the piecewise-parabolic method (PPM) (see [here](Tutorial-GiRaFFE_NRPy-PPM.ipynb)); in this module, we will assume that that step has already been done. Metric quantities are assumed to have been interpolated to cell faces, as is done in [this](Tutorial-GiRaFFE_NRPy-Metric_Face_Values.ipynb) tutorial. 

T&oacute;th's [paper](https://www.sciencedirect.com/science/article/pii/S0021999100965197?via%3Dihub), Eqs. 30 and 31, are one of the first implementations of such a scheme. The original GiRaFFE used a 2D version of the algorithm from [Del Zanna, et al. (2002)](https://arxiv.org/abs/astro-ph/0210618); but since we are not using staggered grids, we can greatly simplify this algorithm with respect to the version used in the original `GiRaFFE`. Instead, we will adapt the implementations of the algorithm used in [Mewes, et al. (2020)](https://arxiv.org/abs/2002.06225) and [Giacomazzo, et al. (2011)](https://arxiv.org/abs/1009.2468), Eqs. 3-11. 

We first write the flux contribution to the induction equation RHS as 
$$
\partial_t A_i = -E_i,
$$
where the electric field $E_i$ is given in ideal MHD (of which FFE is a subset) as
$$
-E_i = \epsilon_{ijk} v^j B^k,
$$
where $v^i$ is the drift velocity,  $B^i$ is the magnetic field, and $\epsilon_{ijk} = \sqrt{\gamma} [ijk]$ is the Levi-Civita tensor.
In Cartesian coordinates, 
\begin{align}
-E_x &= [F^y(B^z)]_x = -[F^z(B^y)]_x \\
-E_y &= [F^z(B^x)]_y = -[F^x(B^z)]_y \\
-E_z &= [F^x(B^y)]_z = -[F^y(B^x)]_z, \\
\end{align}
where 
$$
[F^i(B^j)]_k = \sqrt{\gamma} (v^i B^j - v^j B^i).
$$
To compute the actual contribution to the RHS in some direction $i$, we average the above listed field as calculated on the $+j$, $-j$, $+k$, and $-k$ faces. That is, at some point $(i,j,k)$ on the grid,
\begin{align}
-E_x(x_i,y_j,z_k) &= \frac{1}{4} \left( [F_{\rm HLL}^y(B^z)]_{x(i,j+1/2,k)}+[F_{\rm HLL}^y(B^z)]_{x(i,j-1/2,k)}-[F_{\rm HLL}^z(B^y)]_{x(i,j,k+1/2)}-[F_{\rm HLL}^z(B^y)]_{x(i,j,k-1/2)} \right) \\
-E_y(x_i,y_j,z_k) &= \frac{1}{4} \left( [F_{\rm HLL}^z(B^x)]_{y(i,j,k+1/2)}+[F_{\rm HLL}^z(B^x)]_{y(i,j,k-1/2)}-[F_{\rm HLL}^x(B^z)]_{y(i+1/2,j,k)}-[F_{\rm HLL}^x(B^z)]_{y(i-1/2,j,k)} \right) \\
-E_z(x_i,y_j,z_k) &= \frac{1}{4} \left( [F_{\rm HLL}^x(B^y)]_{z(i+1/2,j,k)}+[F_{\rm HLL}^x(B^y)]_{z(i-1/2,j,k)}-[F_{\rm HLL}^y(B^x)]_{z(i,j+1/2,k)}-[F_{\rm HLL}^y(B^x)]_{z(i,j-1/2,k)} \right). \\
\end{align}
Note the use of $F_{\rm HLL}$ here. This change signifies that the quantity output here is from the HLLE Riemann solver. Note also the indices on the fluxes. Values of $\pm 1/2$ indicate that these are computed on cell faces using the reconstructed values of $v^i$ and $B^i$ and the interpolated values of the metric gridfunctions. So, 
$$
F_{\rm HLL}^i(B^j) = \frac{c_{\rm min} F_{\rm R}^i(B^j) + c_{\rm max} F_{\rm L}^i(B^j) - c_{\rm min} c_{\rm max} (B_{\rm R}^j-B_{\rm L}^j)}{c_{\rm min} + c_{\rm max}}.
$$

The speeds $c_\min$ and $c_\max$ are characteristic speeds that waves can travel through the plasma. In GRFFE, the expressions defining them reduce a function of only the metric quantities. $c_\min$ is the negative of the minimum amongst the speeds $c_-$ and $0$ and $c_\max$ is the maximum amongst the speeds $c_+$ and $0$. The speeds $c_\pm = \left. \left(-b \pm \sqrt{b^2-4ac}\right)\middle/ \left(2a\right) \right.$ must be calculated on both the left and right faces, where 
$$a = 1/\alpha^2,$$ 
$$b = 2 \beta^i / \alpha^2$$
and $$c = g^{ii} - (\beta^i)^2/\alpha^2.$$
An outline of a general finite-volume method is as follows, with the current step in bold:
1. The Reconstruction Step - Piecewise Parabolic Method
    1. Within each cell, fit to a function that conserves the volume in that cell using information from the neighboring cells
        * For PPM, we will naturally use parabolas
    1. Use that fit to define the state at the left and right interface of each cell
    1. Apply a slope limiter to mitigate Gibbs phenomenon
1. Interpolate the value of the metric gridfunctions on the cell faces
1. **Solve the Riemann Problem - Harten, Lax, von Leer, Einfeldt(This notebook, $E_i$ only)**
    1. **Use the left and right reconstructed states to calculate the unique state at boundary**

We will assume in this notebook that the reconstructed velocities and magnetic fields are available on cell faces as input. We will also assume that the metric gridfunctions have been interpolated on the cell faces. 

Solving the Riemann problem, then, consists of two substeps: First, we compute the flux through each face of the cell. Then, we add the average of these fluxes to the right-hand side of the evolution equation for the vector potential. 

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

0. [Step 1](#prelim): Preliminaries
1. [Step 2](#code): Write the C code
    1. [Step 2.a](#hydro_speed): GRFFE characteristic wave speeds
    1. [Step 2.b](#fluxes): Compute the HLLE fluxes
1. [Step 3](#code_validation): Code Validation
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just sets up a subdirectory within `GiRaFFE_standalone_Ccodes/` to which we will write the C code.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
Ccodesdir = "GiRaFFE_standalone_Ccodes/RHSs"
cmd.mkdir(os.path.join(Ccodesdir))

<a id='code'></a>

# Step 2: Write the C code \[Back to [top](#toc)\]
$$\label{code}$$

Recall that there were three equations that we need to code; note also that they are all cyclic permutations of each other. Thus, we only need to code one of them, as we can simply permute the inputs into the function to calculate all three equations. We will use the $E_z$ equation:
$$
-E_z(x_i,y_j,z_k) = \frac{1}{4} \left( [F_{\rm HLL}^x(B^y)]_{z(i+1/2,j,k)}+[F_{\rm HLL}^x(B^y)]_{z(i-1/2,j,k)}-[F_{\rm HLL}^y(B^x)]_{z(i,j+1/2,k)}-[F_{\rm HLL}^y(B^x)]_{z(i,j-1/2,k)} \right),
$$
where 
$$
-E_z = [F^x(B^y)]_z = -[F^y(B^x)]_z
$$
and
$$
[F^i(B^j)]_k = \sqrt{\gamma} (v^i B^j - v^j B^i).
$$

<a id='hydro_speed'></a>

## Step 2.a: GRFFE characteristic wave speeds \[Back to [top](#toc)\]
$$\label{hydro_speed}$$

Next, we will find the speeds at which the hydrodynamics waves propagate. We start from the speed of light (since FFE deals with very diffuse plasmas), which is $c=1.0$ in our chosen units. We then find the speeds $c_+$ and $c_-$ on each face with the function `find_cp_cm`; then, we find minimum and maximum speeds possible from among those.



Below is the source code for `find_cp_cm`, edited to work with the NRPy+ version of GiRaFFE. One edit we need to make in particular is to the term `psim4*gupii` in the definition of `c`; that was written assuming the use of the conformal metric $\tilde{g}^{ii}$. Since we are not using that here, and are instead using the ADM metric, we should not multiply by $\psi^{-4}$.

```c
static inline void find_cp_cm(REAL &cplus,REAL &cminus,const REAL v02,const REAL u0,
                              const REAL vi,const REAL lapse,const REAL shifti,
                              const REAL gammadet,const REAL gupii) {
  const REAL u0_SQUARED=u0*u0;
  const REAL ONE_OVER_LAPSE_SQUARED = 1.0/(lapse*lapse);
  // sqrtgamma = psi6 -> psim4 = gammadet^(-1.0/3.0)
  const REAL psim4 = pow(gammadet,-1.0/3.0);
  //Find cplus, cminus:
  const REAL a = u0_SQUARED * (1.0-v02) + v02*ONE_OVER_LAPSE_SQUARED;
  const REAL b = 2.0* ( shifti*ONE_OVER_LAPSE_SQUARED * v02 - u0_SQUARED * vi * (1.0-v02) );
  const REAL c = u0_SQUARED*vi*vi * (1.0-v02) - v02 * ( gupii -
                                                               shifti*shifti*ONE_OVER_LAPSE_SQUARED);
  REAL detm = b*b - 4.0*a*c;
  //ORIGINAL LINE OF CODE:
  //if(detm < 0.0) detm = 0.0;
  //New line of code (without the if() statement) has the same effect:
  detm = sqrt(0.5*(detm + fabs(detm))); /* Based on very nice suggestion from Roland Haas */
  
  cplus = 0.5*(detm-b)/a;
  cminus = -0.5*(detm+b)/a;
  if (cplus < cminus) {
    const REAL cp = cminus;
    cminus = cplus;
    cplus = cp;
  }
}
```
Comments documenting this have been excised for brevity, but are reproduced in $\LaTeX$ [below](#derive_speed).

We could use this code directly, but there's substantial improvement we can make by changing the code into a NRPyfied form. Note the `if` statement; NRPy+ does not know how to handle these, so we must eliminate it if we want to leverage NRPy+'s full power. (Calls to `fabs()` are also cheaper than `if` statements.) This can be done if we rewrite this, taking inspiration from the other eliminated `if` statement documented in the above code block:
```c
  cp = 0.5*(detm-b)/a;
  cm = -0.5*(detm+b)/a;
  cplus  = 0.5*(cp+cm+fabs(cp-cm));
  cminus = 0.5*(cp+cm-fabs(cp-cm));
```
This can be simplified further, by substituting `cp` and `cm` into the below equations and eliminating terms as appropriate. First note that `cp+cm = -b/a` and that `cp-cm = detm/a`. Thus,
```c
  cplus  = 0.5*(-b/a + fabs(detm/a));
  cminus = 0.5*(-b/a - fabs(detm/a));
```
This fulfills the original purpose of the `if` statement in the original code because we have guaranteed that $c_+ \geq c_-$.

This leaves us with an expression that can be much more easily NRPyfied. So, we will rewrite the following in NRPy+, making only minimal changes to be proper Python. However, it turns out that we can make this even simpler. In GRFFE, $v_0^2$ is guaranteed to be exactly one. In GRMHD, this speed was calculated as $$v_{0}^{2} = v_{\rm A}^{2} + c_{\rm s}^{2}\left(1-v_{\rm A}^{2}\right),$$ where the Alfv&eacute;n speed $v_{\rm A}^{2}$ $$v_{\rm A}^{2} = \frac{b^{2}}{\rho_{b}h + b^{2}}.$$ So, we can see that when the density $\rho_b$ goes to zero, $v_{0}^{2} = v_{\rm A}^{2} = 1$. Then 
\begin{align}
a &= (u^0)^2 (1-v_0^2) + v_0^2/\alpha^2 \\
&= 1/\alpha^2 \\
b &= 2 \left(\beta^i v_0^2 / \alpha^2 - (u^0)^2 v^i (1-v_0^2)\right) \\
&= 2 \beta^i / \alpha^2 \\
c &= (u^0)^2 (v^i)^2 (1-v_0^2) - v_0^2 \left(\gamma^{ii} - (\beta^i)^2/\alpha^2\right) \\
&= -\gamma^{ii} + (\beta^i)^2/\alpha^2,
\end{align}
are simplifications that should save us some time; we can see that $a \geq 0$ is guaranteed. Note that we also force `detm` to be positive. Thus, `detm/a` is guaranteed to be positive itself, rendering the calls to `nrpyAbs()` superfluous. Furthermore, we eliminate any dependence on the Valencia 3-velocity and the time component of the four-velocity, $u^0$. This leaves us free to solve the quadratic in the familiar way: $$c_\pm = \frac{-b \pm \sqrt{b^2-4ac}}{2a}$$.

In flat spacetime, where $\alpha=1$, $\beta^i=0$, and $\gamma^{ij} = \delta^{ij}$, $c_+ > 0$ and $c_- < 0$. For the HLLE solver, we will need both `cmax` and `cmin` to be positive; we also want to choose the speed that is larger in magnitude because overestimating the characteristic speeds will help damp unwanted oscillations. (However, in GRFFE, we only get one $c_+$ and one $c_-$, so we only need to fix the signs here.) 

We will now write a function in C similar to the one used in the old `GiRaFFE`. Notice that since we eliminated the dependence on velocities, none of the input quantities are different on either side of the face. While the original `GiRaFFE` used two functions, the simplifications we've made will allow us to get away with only one without obfuscating anything, with the only downside being that it's a little less obvious how this is related to the GRMHD implementation. We use the same technique as above to replace the `if` statements inherent to the `MAX()` and `MIN()` functions.

This function requires the inverse metric; the following code will easily generate all the needed expressions:
```
from outputC import outputC      # NRPy+: Core C code output module
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
gammaDD = ixp.declarerank2("gammaDD","sym01",DIM=3)
gammaUU,gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
outputC([gammaUU[0][0],gammaUU[1][1],gammaUU[2][2]],["gammaUU00","gammaUU11","gammaUU22"])
```
We modify the output slightly, by putting the final expressions for `gammaUU00`, `gammaUU11`, and `gammaUU22` in a switch statement and changing the assignment to a common variable `gammaUUii`, allowing us to only compute the value needed. 

In [2]:
from outputC import outputC # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import GiRaFFE_NRPy.GiRaFFE_NRPy_Characteristic_Speeds as chsp

gammaDD = ixp.declarerank2("gammaDD","sym01",DIM=3)
betaU = ixp.declarerank1("betaU",DIM=3)
alpha = sp.sympify("alpha")

for flux_dirn in range(3):
    chsp.find_cmax_cmin(flux_dirn,gammaDD,betaU,alpha)
    Ccode_kernel = outputC([chsp.cmax,chsp.cmin],["cmax","cmin"],"returnstring",params="outCverbose=False,CSE_sorting=none")
    Ccode_kernel = Ccode_kernel.replace("cmax","*cmax").replace("cmin","*cmin")
    Ccode_kernel = Ccode_kernel.replace("betaU0","betaUi").replace("betaU1","betaUi").replace("betaU2","betaUi")

    with open(os.path.join(Ccodesdir,"compute_cmax_cmin_dirn"+str(flux_dirn)+".h"),"w") as file:
        file.write(Ccode_kernel)

In [3]:
%%writefile $Ccodesdir/calculate_E_field_flat_all_in_one.h
void find_cmax_cmin(const REAL gammaDD00, const REAL gammaDD01, const REAL gammaDD02,
                    const REAL gammaDD11, const REAL gammaDD12, const REAL gammaDD22,
                    const REAL betaUi, const REAL alpha, const int flux_dirn,
                    REAL *cmax, REAL *cmin) {
    switch(flux_dirn) {
        case 0:
#include "compute_cmax_cmin_dirn0.h"
            break;
        case 1:
#include "compute_cmax_cmin_dirn1.h"
            break;
        case 2:
#include "compute_cmax_cmin_dirn2.h"
            break;
        default:
            printf("Invalid parameter flux_dirn!"); *cmax = 1.0/0.0; *cmin = 1.0/0.0;
            break;
    }
}

Overwriting GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


<a id='fluxes'></a>

## Step 2.b: Compute the HLLE fluxes \[Back to [top](#toc)\]
$$\label{fluxes}$$

First, we write a standard HLLE solver based on eq. 3.15 in [the HLLE paper](https://epubs.siam.org/doi/pdf/10.1137/1025002),
$$
F^{\rm HLL} = \frac{c_{\rm min} F_{\rm R} + c_{\rm max} F_{\rm L} - c_{\rm min} c_{\rm max} (U_{\rm R}-U_{\rm L})}{c_{\rm min} + c_{\rm max}}
$$

In [4]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

REAL HLLE_solve(REAL F0B1_r, REAL F0B1_l, REAL U_r, REAL U_l, REAL cmin, REAL cmax) {
  // Eq. 3.15 of https://epubs.siam.org/doi/abs/10.1137/1025002?journalCode=siread
  // F_HLLE = (c_min F_R + c_max F_L - c_min c_max (U_R-U_L)) / (c_min + c_max)
  return (cmin*F0B1_r + cmax*F0B1_l - cmin*cmax*(U_r-U_l)) / (cmin+cmax);
}


Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


Here, we will write declare the function that will compute the fluxes and pass them to the HLLE solver, then add that result to the $A_i$ right-hand side. We pass the inputs necessary for to compute $E_z$. This includes the standard NRPy+ `params`, as well as pointers to the needed components of the Valencia three-velocity and magnetic field; note that `Br1=Brflux_dirn` and `Bl1=Blflux_dirn` must be set that way to get correct results. We also pass the output `A2_rhs` (again, this will be changed to reflect the actual component we want to calculate) and parameters `SIGN` ($\pm 1$) and flux_dirn (`0`, `1`, or `2`, needed to compute the offsets for memory access). 

With the `#include`, we set our parameters as usual; if the declaration for a variable cannot be found elsewhere in the code, it was defined here.

In [5]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

/*
Calculate the electric flux on both faces in the input direction.
The input count is an integer that is either 0 or 1. If it is 0, this implies
that the components are input in order of a backwards permutation  and the final
results will need to be multiplied by -1.0. If it is 1, then the permutation is forwards.
 */
void calculate_E_field_flat_all_in_one(const paramstruct *params,
                                       const REAL *Vr0,const REAL *Vr1,
                                       const REAL *Vl0,const REAL *Vl1,
                                       const REAL *Br0,const REAL *Br1,
                                       const REAL *Bl0,const REAL *Bl1,
                                       const REAL *Brflux_dirn,
                                       const REAL *Blflux_dirn,
                                       const REAL *gamma_faceDD00, const REAL *gamma_faceDD01, const REAL *gamma_faceDD02,
                                       const REAL *gamma_faceDD11, const REAL *gamma_faceDD12, const REAL *gamma_faceDD22,
                                       const REAL *beta_faceU0, const REAL *beta_faceU1, const REAL *alpha_face,
                                       REAL *A2_rhs,const REAL SIGN,const int flux_dirn) {
    // This function is written to be generic and compute the contribution for all three AD RHSs.
    // However, for convenience, the notation used in the function itself is for the contribution
    // to AD2, specifically the [F_HLL^x(B^y)]_z term, with reconstructions in the x direction. This
    // corresponds to flux_dirn=0 and count=1 (which corresponds to SIGN=+1.0).
    // Thus, Az(i,j,k) += 0.25 ( [F_HLL^x(B^y)]_z(i+1/2,j,k)+[F_HLL^x(B^y)]_z(i-1/2,j,k)) are solved here.
    // The other terms are computed by cyclically permuting the indices when calling this function.
#include "../set_Cparameters.h"


Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


Again, we are going to use the notation for the RHS of `A_z` (or `AD2`), specifically the $[F_{\rm HLL}^x(B^y)]_{z(i+1/2,j,k)}+[F_{\rm HLL}^x(B^y)]_{z(i-1/2,j,k)}$ terms, corresponding to `flux_dirn=0` and `count=1` (which itself corresponds to `SIGN=1.0`). Specifically, `flux_dirn=0,1,2` is determined by which of $i,j,k$ is offset by $\pm 1/2$, respectively, and `SIGN` corresponds to the sign in from the $F^i(B^j)$ terms, which is determined by the permutation of $(i,j,k)$ represented by $-E_k = F^i(B^j)$.

We begin a straightforward loop over the grid interior; then, we must read in all necessary input values from memory. 
Now, recall that, when reconstructing the $x$-direction, we reconstructed to the $i-1/2$ face, or $(i-1/2,j,k)$ for a memory location corresponding to `i,j,k`. So, since we must read in values corresponding to $(i-1/2,j,k)$ and $(i+1/2,j,k)$, we read from `i,j,k` and `i+1,j,k`, respectively, permuting as necessary dictated by the input value of `flux_dirn`.

In [6]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

#pragma omp parallel for
    for(int i2=NGHOSTS; i2<NGHOSTS+Nxx2; i2++) {
        for(int i1=NGHOSTS; i1<NGHOSTS+Nxx1; i1++) {
            for(int i0=NGHOSTS; i0<NGHOSTS+Nxx0; i0++) {
                // First, we set the index from which we will read memory. indexp1 is incremented by
                // one point in the direction of reconstruction. These correspond to the faces at at
                // i-1/2 and i+1/2, respectively.

                // Now, we read in memory. We need the x and y components of velocity and magnetic field on both
                // the left and right sides of the interface at *both* faces.
                // Here, the point (i0,i1,i2) corresponds to the point (i-1/2,j,k)
                const int index           = IDX3S(i0,i1,i2);
                const double alpha        = alpha_face[index];
                const double betaU0       = beta_faceU0[index];
                const double betaU1       = beta_faceU1[index];
                const double v_rU0        = alpha*Vr0[index]-betaU0;
                const double v_rU1        = alpha*Vr1[index]-betaU1;
                const double B_rU0        = Br0[index];
                const double B_rU1        = Br1[index];
                const double B_rflux_dirn = Brflux_dirn[index];
                const double v_lU0        = alpha*Vl0[index]-betaU0;
                const double v_lU1        = alpha*Vl1[index]-betaU1;
                const double B_lU0        = Bl0[index];
                const double B_lU1        = Bl1[index];
                const double B_lflux_dirn = Blflux_dirn[index];
                // We will also need need the square root of the metric determinant here at this point:
                const REAL gxx = gamma_faceDD00[index];
                const REAL gxy = gamma_faceDD01[index];
                const REAL gxz = gamma_faceDD02[index];
                const REAL gyy = gamma_faceDD11[index];
                const REAL gyz = gamma_faceDD12[index];
                const REAL gzz = gamma_faceDD22[index];
                const REAL sqrtgammaDET = sqrt( gxx*gyy*gzz
                                             -  gxx*gyz*gyz
                                             +2*gxy*gxz*gyz
                                             -  gyy*gxz*gxz
                                             -  gzz*gxy*gxy );

                // *******************************
                // REPEAT ABOVE, but at i+1, which corresponds to point (i+1/2,j,k)
                //     Recall that the documentation here assumes flux_dirn==0, but the
                //     algorithm is generalized so that any flux_dirn or velocity/magnetic
                //     field component can be computed via permuting the inputs into this
                //     function.
                const int indexp1            = IDX3S(i0+(flux_dirn==0),i1+(flux_dirn==1),i2+(flux_dirn==2));
                const double alpha_p1        = alpha_face[indexp1];
                const double betaU0_p1       = beta_faceU0[indexp1];
                const double betaU1_p1       = beta_faceU1[indexp1];
                const double v_rU0_p1        = alpha_p1*Vr0[indexp1]-betaU0_p1;
                const double v_rU1_p1        = alpha_p1*Vr1[indexp1]-betaU1_p1;
                const double B_rU0_p1        = Br0[indexp1];
                const double B_rU1_p1        = Br1[indexp1];
                const double B_rflux_dirn_p1 = Brflux_dirn[indexp1];
                const double v_lU0_p1        = alpha_p1*Vl0[indexp1]-betaU0_p1;
                const double v_lU1_p1        = alpha_p1*Vl1[indexp1]-betaU1_p1;
                const double B_lU0_p1        = Bl0[indexp1];
                const double B_lU1_p1        = Bl1[indexp1];
                const double B_lflux_dirn_p1 = Blflux_dirn[indexp1];
                // We will also need need the square root of the metric determinant here at this point:
                const REAL gxx_p1 = gamma_faceDD00[indexp1];
                const REAL gxy_p1 = gamma_faceDD01[indexp1];
                const REAL gxz_p1 = gamma_faceDD02[indexp1];
                const REAL gyy_p1 = gamma_faceDD11[indexp1];
                const REAL gyz_p1 = gamma_faceDD12[indexp1];
                const REAL gzz_p1 = gamma_faceDD22[indexp1];
                const REAL sqrtgammaDET_p1 = sqrt( gxx_p1*gyy_p1*gzz_p1
                                                -  gxx_p1*gyz_p1*gyz_p1
                                                +2*gxy_p1*gxz_p1*gyz_p1
                                                -  gyy_p1*gxz_p1*gxz_p1
                                                -  gzz_p1*gxy_p1*gxy_p1 );

                // *******************************

                // DEBUGGING:
//                 if(flux_dirn==0 && SIGN>0 && i1==Nxx_plus_2NGHOSTS1/2 && i2==Nxx_plus_2NGHOSTS2/2) {
//                     printf("index=%d & indexp1=%d\n",index,indexp1);
//                 }


Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


Here, we we calculate the flux and state vectors for the electric field. The flux vector is here given as 
$$
[F^i(B^j)]_k = \sqrt{\gamma} (v^i B^j - v^j B^i),
$$
or
$$
[F^x(B^y)]_z = \sqrt{\gamma} (v^x B^y - v^y B^x)
$$
in our specific case.
Here, $v^i$ is the drift velocity and $B^i$ is the magnetic field.

The state vector is simply the magnetic field component specified by the notation $[F^i(B^j)]_k$ (that is, $j$). Here, then, the notation $[F^x(B^y)]_z$ dictates the use of $B^y$. The way we shuffle inputs to produce the other terms we will need conveniently guarantees that by using the $B^y$ component here, we always choose the correct component in other cases.

In [7]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

                // Since we are computing A_z, the relevant equation here is:
                // -E_z(x_i,y_j,z_k) = 0.25 ( [F_HLL^x(B^y)]_z(i+1/2,j,k)+[F_HLL^x(B^y)]_z(i-1/2,j,k)
                //                           -[F_HLL^y(B^x)]_z(i,j+1/2,k)-[F_HLL^y(B^x)]_z(i,j-1/2,k) )
                // We will construct the above sum one half at a time, first with SIGN=+1, which
                // corresponds to flux_dirn = 0, count=1, and
                //  takes care of the terms:
                //  [F_HLL^x(B^y)]_z(i+1/2,j,k)+[F_HLL^x(B^y)]_z(i-1/2,j,k)

                // ( Note that we will repeat the above with flux_dirn = 1, count = 0, with SIGN=-1
                //   AND with the input components switched (x->y,y->x) so that we get the term
                // -[F_HLL^y(B^x)]_z(i,j+1/2,k)-[F_HLL^y(B^x)]_z(i,j-1/2,k)
                // thus completing the above sum. )

                // Here, [F_HLL^i(B^j)]_k = (v^i B^j - v^j B^i) in general.

                // Calculate the flux vector on each face for each component of the E-field:
                // The F(B) terms are as Eq. 6 in Giacomazzo: https://arxiv.org/pdf/1009.2468.pdf
                // [F^i(B^j)]_k = \sqrt{\gamma} (v^i B^j - v^j B^i)
                // Therefore since we want [F_HLL^x(B^y)]_z,
                // we will code     (v^x           B^y   - v^y           B^x) on both left and right faces.
                const REAL F0B1_r = sqrtgammaDET*(v_rU0*B_rU1 - v_rU1*B_rU0);
                const REAL F0B1_l = sqrtgammaDET*(v_lU0*B_lU1 - v_lU1*B_lU0);

                // Compute the state vector for these terms:
                const REAL U_r = B_rflux_dirn;
                const REAL U_l = B_lflux_dirn;


Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


Finally, we call the HLLE solver we wrote earlier.

In [8]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

                REAL cmin,cmax;
                // Basic HLLE solver:
                find_cmax_cmin(gxx,gxy,gxz,
                               gyy,gyz,gzz,
                               betaU0,alpha,flux_dirn,
                               &cmax, &cmin);
                const REAL FHLL_0B1 = HLLE_solve(F0B1_r, F0B1_l, U_r, U_l, cmin, cmax);


Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


Then, we set the flux and state vectors as before and pass them to the HLLE solver, but at the point `i+1`.

In [9]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

                // ************************************
                // ************************************
                // REPEAT ABOVE, but at point i+1
                // Calculate the flux vector on each face for each component of the E-field:
                const REAL F0B1_r_p1 = sqrtgammaDET_p1*(v_rU0_p1*B_rU1_p1 - v_rU1_p1*B_rU0_p1);
                const REAL F0B1_l_p1 = sqrtgammaDET_p1*(v_lU0_p1*B_lU1_p1 - v_lU1_p1*B_lU0_p1);

                // Compute the state vector for this flux direction
                const REAL U_r_p1 = B_rflux_dirn_p1;
                const REAL U_l_p1 = B_lflux_dirn_p1;
                //const REAL U_r_p1 = B_rU1_p1;
                //const REAL U_l_p1 = B_lU1_p1;
                // Basic HLLE solver, but at the next point:
                find_cmax_cmin(gxx_p1,gxy_p1,gxz_p1,
                               gyy_p1,gyz_p1,gzz_p1,
                               betaU0_p1,alpha_p1,flux_dirn,
                               &cmax, &cmin);
                const REAL FHLL_0B1p1 = HLLE_solve(F0B1_r_p1, F0B1_l_p1, U_r_p1, U_l_p1, cmin, cmax);
                // ************************************
                // ************************************



Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


With the Riemann problem solved, we add the contributions to the RHSs. Here is where we multiply by the input `SIGN`; note that we also multiply by $1/4$, because the algorithm calls for an average of the four terms. That is, our final value for $-E_z$ will be the average of that calculated at the $\pm x$ and $\pm y$ faces of the cell.

Finally, we close out our `for` loops.

In [10]:
%%writefile -a $Ccodesdir/calculate_E_field_flat_all_in_one.h

                // With the Riemann problem solved, we add the contributions to the RHSs:
                // -E_z(x_i,y_j,z_k) &= 0.25 ( [F_HLL^x(B^y)]_z(i+1/2,j,k)+[F_HLL^x(B^y)]_z(i-1/2,j,k)
                //                            -[F_HLL^y(B^x)]_z(i,j+1/2,k)-[F_HLL^y(B^x)]_z(i,j-1/2,k) )
                // (Eq. 11 in https://arxiv.org/pdf/1009.2468.pdf)
                // This code, as written, solves the first two terms for flux_dirn=0. Calling this function for count=0
                // and flux_dirn=1 flips x for y to solve the latter two, switching to SIGN=-1 as well.

                // Here, we finally add together the output of the HLLE solver at i-1/2 and i+1/2
                // We also multiply by the SIGN dictated by the order of the input vectors and divide by 4.
                A2_rhs[index] += SIGN*0.25*(FHLL_0B1 + FHLL_0B1p1);
                // flux dirn = 0 ===================>   i-1/2       i+1/2
                //               Eq 11 in Giacomazzo:
                //               -FxBy(avg over i-1/2 and i+1/2) + FyBx(avg over j-1/2 and j+1/2)
                //               Eq 6 in Giacomazzo:
                //               FxBy = vxBy - vyBx
                //             ->
                //               FHLL_0B1 = vyBx - vxBy

            } // END LOOP: for(int i0=NGHOSTS; i0<NGHOSTS+Nxx0; i0++)
        } // END LOOP: for(int i1=NGHOSTS; i1<NGHOSTS+Nxx1; i1++)
    } // END LOOP: for(int i2=NGHOSTS; i2<NGHOSTS+Nxx2; i2++)
}


Appending to GiRaFFE_standalone_Ccodes/RHSs/calculate_E_field_flat_all_in_one.h


<a id='code_validation'></a>

# Step 3: Code Validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are generated by the python module


In [11]:
# Define the directory that we wish to validate against:
valdir = "GiRaFFE_NRPy/GiRaFFE_Ccode_library/RHSs/"

import GiRaFFE_NRPy.GiRaFFE_NRPy_Afield_flux_handwritten as Af
Af.GiRaFFE_NRPy_Afield_flux(valdir)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare
files = ["calculate_E_field_flat_all_in_one.h"]

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file calculate_E_field_flat_all_in_one.h
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy-Afield_flux.pdf](Tutorial-GiRaFFE_NRPy-Afield_flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [12]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy-Afield_flux",location_of_template_file=os.path.join(".."))

pdflatex: security risk: running with elevated privileges
pdflatex: security risk: running with elevated privileges
pdflatex: security risk: running with elevated privileges
Created Tutorial-GiRaFFE_NRPy-Afield_flux.tex, and compiled LaTeX file to
    PDF file Tutorial-GiRaFFE_NRPy-Afield_flux.pdf
